In [2]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
#!pip install matplotlib

In [4]:
from pathlib import Path
from dataclasses import dataclass
import yaml


In [5]:
@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    data_load_path: Path
    local_data_path: Path
    status_file: Path
    all_required_files: list


In [6]:
pwd

'c:\\Users\\RSR\\PYTHON\\LinearRegression\\Research'

In [7]:
os.chdir('c:\\Users\\RSR\\PYTHON\\LinearRegression\\src')

In [8]:
from CancerML.Utils.common import *
from CancerML.Constants import *

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=Read_Yaml(Path("Config/Config.yaml"))
        self.params=Read_Yaml(params_filepath)

        
        create_directories([self.config.artifacts_root])

    def Get_DataPreProcesing_Config(self)->DataPreprocessingConfig:
        config=self.config.data_preprocessing

        create_directories([config.root_dir])

        Data_PreProcessing_config=DataPreprocessingConfig(
            root_dir=config.root_dir,
            data_load_path=config.data_load_path,
            local_data_path=config.local_data_path,
            status_file=config.status_file,
            all_required_files=config.all_required_files
        )
        return Data_PreProcessing_config
    


In [ ]:
class DataPreProcessing:
    def __init__(self,config:DataPreprocessingConfig):
        self.config=config
    
    def Check_files(self)->bool:
        file=Path(self.config.data_load_path)
        if file.name  in self.config.all_required_files:
            validation_status=True
            with open(self.config.status_file,'w') as f:
                f.write(f"Validation Status : {validation_status}")
                print(f"file existed : {file.name}")
            
            
        else:
            validation_status=False
            print(validation_status)

        return validation_status
    
    def DataFrame(self):
        file='cancer_dataset.csv'
        if file not in self.config.all_required_files:
            logger.info(f"Data set does not exist in the source path : please check")



In [14]:
try:
    config=ConfigurationManager()
    data_preprocess=config.Get_DataPreProcesing_Config()
    data_preprocessing=DataPreProcessing(config=data_preprocess)
    check_file=data_preprocessing.Check_files()
    

except Exception as e:
    raise e

[2025-12-04 13:07:18,448: INFO: common: YAML loaded successfully: Config\Config.yaml]
[2025-12-04 13:07:18,467: INFO: common: YAML loaded successfully: params.yaml]
[2025-12-04 13:07:18,469: INFO: common: Directory created: artifacts]
[2025-12-04 13:07:18,474: INFO: common: Directory created: artifacts/data_preprocessing]
